In [1]:
import pandas as pd
sales_df=pd.read_csv('sales-data.csv')
cogs_df=pd.read_csv('cogs-data.csv')
opex_df=pd.read_csv('operational-expenses.csv')
print("Sales Data:")
display(sales_df.head())
print("\nCOGS Data:")
display(cogs_df.head())
print("\nOperational Expenses Data:")
display(opex_df.head())

Sales Data:


,OrderID,Date,SKU,Quantity,UnitPrice,TotalRevenue
0,ORD001,2024-08-01,SKU123,2,35,70
1,ORD002,2024-08-01,SKU124,1,55,55
2,ORD003,2024-08-02,SKU125,3,28,84
3,ORD004,2024-08-02,SKU123,1,35,35
4,ORD005,2024-08-02,SKU127,5,20,100



COGS Data:


,SKU,COGS
0,SKU123,20
1,SKU124,15
2,SKU125,25
3,SKU127,22
4,SKU128,18



Operational Expenses Data:


,Date,ExpenseCategory,Amount,SKU,Notes
0,2024-08-01,Rent,2000,NaN,warehouserent
1,2024-08-01,Shipping,550,SKU125,shipping costs
2,2024-08-01,salaries,5000,NaN,staff salaries
3,2024-08-02,utilities,300,NaN,office utilities
4,2024-08-03,Marketing,800,SKU123,email marketing


In [2]:
merged_df=pd.merge(sales_df, cogs_df, on='SKU', how='left')
merged_df["GrossProfit"]=(merged_df["UnitPrice"]-merged_df["COGS"])*merged_df["Quantity"]
display(merged_df.head())

,OrderID,Date,SKU,Quantity,UnitPrice,TotalRevenue,COGS,GrossProfit
0,ORD001,2024-08-01,SKU123,2,35,70,20,30
1,ORD002,2024-08-01,SKU124,1,55,55,15,40
2,ORD003,2024-08-02,SKU125,3,28,84,25,9
3,ORD004,2024-08-02,SKU123,1,35,35,20,15
4,ORD005,2024-08-02,SKU127,5,20,100,22,-10


In [3]:
opex_sum=opex_df.groupby('Date')['Amount'].sum().reset_index()
final_df=pd.merge(merged_df, opex_sum, on='Date', how='left')
final_df["Amount"]=final_df["Amount"].fillna(0)
final_df["NetProfit"]=final_df["GrossProfit"]-final_df["Amount"]
final_df.head()

,OrderID,Date,SKU,Quantity,UnitPrice,TotalRevenue,COGS,GrossProfit,Amount,NetProfit
0,ORD001,2024-08-01,SKU123,2,35,70,20,30,7550.0,-7520.0
1,ORD002,2024-08-01,SKU124,1,55,55,15,40,7550.0,-7510.0
2,ORD003,2024-08-02,SKU125,3,28,84,25,9,300.0,-291.0
3,ORD004,2024-08-02,SKU123,1,35,35,20,15,300.0,-285.0
4,ORD005,2024-08-02,SKU127,5,20,100,22,-10,300.0,-310.0


In [4]:
import plotly.express as px
sku_profit=final_df.groupby('SKU')['GrossProfit'].sum().reset_index()
fig=px.bar(sku_profit, x='SKU', y='GrossProfit', title='Gross Profit by SKU',
           labels={'GrossProfit':'Gross Profit ($)', 'SKU':'SKU'},
           hover_data=['GrossProfit'])
fig.show()

In [5]:
import plotly.express as px
fig=px.line(merged_df, x='Date', y=["TotalRevenue", "COGS"],title="Sales vs COGS Over Time")
fig.show()

In [6]:
summary_df = merged_df.groupby('SKU').agg({
    'TotalRevenue': 'sum',
    'COGS': 'sum',
    'GrossProfit': 'sum'
}).reset_index()

summary_df['GrossMargin (%)'] = (summary_df['GrossProfit'] / summary_df['TotalRevenue']) * 100
summary_df['PerformanceFlag'] = summary_df['GrossProfit'].apply(lambda x: 'Low Profit' if x < 20 else 'OK')

display(summary_df)

,SKU,TotalRevenue,COGS,GrossProfit,GrossMargin (%),PerformanceFlag
0,SKU123,105,40,45,42.857143,OK
1,SKU124,165,30,120,72.727273,OK
2,SKU125,84,25,9,10.714286,Low Profit
3,SKU127,100,22,-10,-10.000000,Low Profit
4,SKU128,240,36,96,40.000000,OK
5,SKU129,140,12,56,40.000000,OK
6,SKU130,132,10,92,69.696970,OK
7,SKU133,85,40,45,52.941176,OK


In [7]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))
plt.bar(summary_df['SKU'], summary_df['GrossProfit'], color='skyblue')
plt.title('Gross Profit by SKU')
plt.xlabel('SKU')
plt.ylabel('Gross Profit')
plt.tight_layout()
plt.savefig('gross_profit_by_sku.png')
plt.close()

In [ ]:
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)
pdf.set_font("Arial", 'B', 16)/
pdf.cell(200, 10, txt="SKU Profitability Report", ln=True, align='C')
pdf.set_font("Arial", size=12)
pdf.ln(10)
pdf.cell(200, 10, txt="Summary KPIs and Automated Insights:", ln=True)
for i, row in summary_df.iterrows():
    pdf.cell(200, 10, txt=f"SKU: {row['SKU']} | GP: {row['GrossProfit']} | Margin: {row['GrossMargin (%)']:.2f}% | {row['PerformanceFlag']}", ln=True)
pdf.ln(10)
if os.path.exists("gross_profit_by_sku.png"):
    pdf.image("gross_profit_by_sku.png", w=170)
pdf.output("SKU_Report.pdf")